In [195]:
def get_contractions():
    contractions = { 
        "ain't": "am not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he's": "he is",
        "how'd": "how did",
        "how'll": "how will",
        "how's": "how is",
        "i'd": "i would",
        "i'll": "i will",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'll": "it will",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "must've": "must have",
        "mustn't": "must not",
        "needn't": "need not",
        "oughtn't": "ought not",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "she'd": "she would",
        "she'll": "she will",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "that'd": "that would",
        "that's": "that is",
        "there'd": "there had",
        "there's": "there is",
        "they'd": "they would",
        "they'll": "they will",
        "they're": "they are",
        "they've": "they have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'll": "we will",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "where'd": "where did",
        "where's": "where is",
        "who'll": "who will",
        "who's": "who is",
        "won't": "will not",
        "wouldn't": "would not",
        "you'd": "you would",
        "you'll": "you will",
        "you're": "you are"
    }
    return contractions;

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from pprint import pprint
from nltk.corpus import stopwords
import sklearn.decomposition
import matplotlib.pyplot as plt
import itertools
import string
import re
from keras.models import model_from_json
from keras.models import load_model
import gensim
import os
import numpy as np
from sklearn.utils import resample
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import word_tokenize
from sklearn.utils import class_weight
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU, GlobalMaxPool1D
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import roc_auc_score
from keras.initializers import Constant





In [95]:
def get_all_data():
	# Load data
	engine = create_engine('mysql://root:@localhost:3306/steam')
	# steam_data_query = """SELECT url AS reviewid, content, CAST(recommend AS SIGNED) AS recommend
	#     FROM latest_review"""
	data_query = """SELECT gameid, url, CAST(recommend AS SIGNED) AS recommend, hours_2w, hours_all, posttime, updatetime, EAG, CAST(compensation AS SIGNED) AS compensation, content, initial_release_date, lang
	FROM latest_review
	WHERE lang = 'en'
	AND (trim(content) != ''
	OR trim(content) != ' '
	OR content IS NOT NULL);
	"""
	data = pd.read_sql(data_query, engine)
# 	print(len(data))
	return data;

In [96]:
data = get_all_data()

In [197]:
def pre_process(text):

    text = text.lower().rstrip('\n').strip()
#     text = text.replace('\'','')
    tokens = word_tokenize(text)
#     text_table = str.maketrans('', '', string.punctuation)
#     text_stripped = [w.translate(text_table) for w in text_tokens]
#     text_words = [word for word in text_stripped if word.isalpha()]
    
#     stop_words = set(stopwords.words('english'))
#     text_words = [w for w in text_words if not w in stop_words]
#     text = " ".join(text_words)
    
    new_text = []
    for word in tokens:
        if word in contractions:
            new_word = contractions[word]
            new_text.append(new_word)
        else:
            new_text.append(word)
    text = " ".join(new_text)
    return text;
       
    
    
    

In [198]:
data['pre'] = data['content'].map(pre_process)


In [133]:
data['pre'][0]
# data['pre'][0]
# X[0]

"it's subjectively a compilation of mediocre minigames. if you don't have a group of friends to play this with, just avoid it."

In [199]:
df_minority = data[data['recommend'] == 0]

df_majority = data[data['recommend'] == 1]
df_majority_downsampled = resample(df_majority, replace = False, n_samples = len(df_minority))
# print(len(df_majority_downsampled))
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

df_downsampled = df_downsampled.reset_index(drop = True)
df_downsampled['recommend'].value_counts()
# len(df_downsampled)


1    13251
0    13251
Name: recommend, dtype: int64

In [209]:
# X = df_downsampled['pre']
# y = df_downsampled['recommend']
X = data['pre']
y = data['recommend']
data['recommend'].value_counts()


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

1    40513
0    13251
Name: recommend, dtype: int64

In [212]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit


skf = StratifiedShuffleSplit(n_splits = 5, test_size = 0.2)
num_folds = skf.get_n_splits(X)


train_accs = []
test_accs = []
auc_accs = []

for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    counter = str(index + 1)
    print("Training on fold " + counter + "/" + str(num_folds) + "...")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    vectorizer = CountVectorizer(binary = 'true')
    classifier = training_step(X_train.values, y_train.values, vectorizer)
    
    train_accs.append(simple_evaluation(X_train.values, y_train.values))
    test_accs.append(simple_evaluation(X_test.values, y_test.values))
    y_pred = classifier.predict(vectorizer.transform(X_test.values))
    auc_accs.append(roc_auc_score(y_test.values, y_pred))
    

Training on fold 1/5...
Training on fold 2/5...
Training on fold 3/5...
Training on fold 4/5...
Training on fold 5/5...


In [214]:
import numpy as np
print(np.mean(train_accs))
print(np.median(train_accs))

print(np.mean(test_accs))
print(np.median(test_accs))

print(np.mean(auc_accs))
print(np.median(auc_accs))

47.15072888330892
47.1832786961475
46.212219845624475
45.894169069096996
0.5075249697875699
0.5074208248051618


In [210]:
# !pip3 install imblearn
from imblearn.combine import SMOTETomek

def training_step(X, y, vectorizer):
    
    X = vectorizer.fit_transform(X)
    
    smote = SMOTETomek(ratio = 'auto')
    X_sm, y_sm = smote.fit_sample(X, y)
#     smote = SMOTE(ratio = 'minority')
#     X_sm

#     training_text = [X for data in data]
#     training_result = [y for data in data]
#     print(training_text)
    

    return BernoulliNB().fit(X_sm, y_sm)



In [189]:

# print(X_train.values)
# print(y_train.values)
classifier = training_step(X_train.values, y_train.values, vectorizer)


In [190]:
# result = classifier.predict(vectorizer.transform(["this is the best game"]))
# print(len(X))
def analyze_text(classifier, vectorizer, text):
    return text, classifier.predict(vectorizer.transform([text]))

def print_result(result):
    text, analysis_result = result
    print_text = "Positive" if str(analysis_result[0]) == '1' else "Negative"
    print(text, ":", print_text)
# print_result(analyze_text(classifier, vectorizer, "Worst movie ever"))



In [194]:
def simple_evaluation(X_test, y_test):
    evaluation_text     = X_test
    evaluation_result   = y_test

    total = len(evaluation_text)
    corrects = 0
    for index in range(0, total):
        analysis_result = analyse_text(classifier, vectorizer, evaluation_text[index])
        text, result = analysis_result
        corrects += 1 if result[0] == evaluation_result[index] else 0

    return corrects * 100 / total



64.49726466704395

In [193]:

print(roc)

0.6464017834137398


In [ ]:
classifier.classes_

In [ ]:
y_pred = 